### Domain generalization evaluation  (see Section 4.2.3.5. of thesis)

This notebook focuses on getting the similarities of each domain to the whole dataset. Further, based on the least similar datasets a seperate training is performed and evaluated, excluding the least similar domains, to evaluate whether the model can generalize well or not (for more details see Section 4.2.3.5.)

Please keep in mind that these notebooks are primarily used for conducting experiments, live coding, and implementing and evaluating the approaches presented in the thesis. As a result, the code in this notebook may not strictly adhere to best practice coding standards.

*Here, training with GPU is required. Thus, either use Google Colab or setup you GPU properly.*

In [ ]:
# ONLY IF USED ON LOCAL VIEW
# only execute once
import os

# Getting the parent directory
os.chdir("..")
os.chdir("..")
os.chdir("..")
os.chdir("..")

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 52.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from numpy import array, argmax

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, TFAutoModel

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

def import_test_train(local):
  """
  This imports the given train and testset locally or not and returns it.

  :param local: If set to true, it will return the trainset from a local view. Otherwise it will open drive mount and attempts to connect to your
  drive folders.
  """

  assert type(local) == bool, f"Type is not valid. Expected boolean, recieved: {type(local)}"

  if local:
    from google.colab import drive
    drive.mount('/content/gdrive')

    df_test = pd.read_csv('/content/gdrive/MyDrive/Experiment/testset_DE_Trigger.csv')
    df_train = pd.read_csv('/content/gdrive/MyDrive/Experiment/trainset_DE_Trigger.csv')

    return df_test, df_train

  else:
    df_test = pd.read_csv('./Experiment/testset_DE_Trigger.csv')
    df_train = pd.read_csv('./Experiment/trainset_DE_Trigger.csv')

    return df_test, df_train

# importing test and trainset
df_test, df_train = import_test_train(True)

# If you want to use it locally, make sure to execute the notebooks from the root directory of this project and uncomment the following line:
# df_test, df_train = import_test_train(False)

# concat the train and testset to get the full dataset
df = df_train.append(df_test)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Text similarity approach

In [ ]:
stop = list(stopwords.words('german'))

# return similarity for each source
def return_source_as_string(df, source):
  df_src = df[df.source == source]
  source_str = "".join(list(df_src.content))

  df_nsrc = df[(df != source).any(axis=1)]
  not_source_str = "".join(list(df_nsrc.content))
  return source_str, not_source_str

# calculate text similarity based on tf idf
def cal_tfidf(df, source):
  res = return_source_as_string(df, source)
  vect = TfidfVectorizer(min_df=1, stop_words = stop)
  tfidf = vect.fit_transform(res)
  pairwise_similarity = tfidf * tfidf.T

  similarity = 100*round(pairwise_similarity.toarray()[0][1],4)

  return source, similarity

In [ ]:
# Results of tf idf
sources = ["crowdflower", "GoEmotions", "Isear",
           "DailyDialog", "Emotion-stimulus", "Tails"]
res = []

for source in sources:
  res.append(cal_tfidf(df, source))

pd.DataFrame(res, columns=["source", "similarity"])

,source,similarity
0,crowdflower,84.02
1,GoEmotions,76.84
2,Isear,63.79
3,DailyDialog,74.26
4,Emotion-stimulus,56.55
5,Tails,45.16


### Regular training with BERT

Here we exclude the least similar Domains. We will just use them for later evaluation, to see if BERT can generalize well.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-uncased")
bert = TFAutoModel.from_pretrained("dbmdz/bert-base-german-uncased")

Some layers from the model checkpoint at dbmdz/bert-base-german-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at dbmdz/bert-base-german-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
df_train = df_train[~df_train['source'].str.contains('Emotion-stimulus')]
df_train.reset_index(drop=True, inplace=True)

# max length of berttokenizer  is 512
max_length=100

#creating mask for tokens
Xids_train=np.zeros((df_train.shape[0],max_length))
Xmask_train=np.zeros((df_train.shape[0],max_length))
y_train=np.zeros((df_train.shape[0],1))

#creating mask for tokens
Xids_test=np.zeros((df_test.shape[0],max_length))
Xmask_test=np.zeros((df_test.shape[0],max_length))

In [ ]:
for i,sequence in enumerate(df_train['content']):
    tokens=tokenizer.encode_plus(sequence,max_length=max_length,padding='max_length',add_special_tokens=True,
                           truncation=True,return_token_type_ids=False,return_attention_mask=True,
                           return_tensors='tf')

    Xids_train[i,:] = tokens['input_ids']
    Xmask_train[i,:] = tokens['attention_mask']
    y_train[i,0] = df_train.loc[i,'label_id']

y_train = to_categorical(y_train)

for i,sequence in enumerate(df_test['content']):
    tokens=tokenizer.encode_plus(sequence,max_length=max_length,padding='max_length',add_special_tokens=True,
                           truncation=True,return_token_type_ids=False,return_attention_mask=True,
                           return_tensors='tf')

    Xids_test[i,:] = tokens['input_ids']
    Xmask_test[i,:] = tokens['attention_mask']

In [ ]:
dataset=tf.data.Dataset.from_tensor_slices((Xids_train,Xmask_train,y_train))

def map_func(input_ids,mask,labels):
    return {'input_ids':input_ids,'attention_mask':mask},labels

dataset=dataset.map(map_func)
dataset=dataset.shuffle(100000).batch(64).prefetch(1000)

DS_size=len(list(dataset))

train=dataset.take(round(DS_size*0.90))
val=dataset.skip(round(DS_size*0.90))

In [ ]:
dataset_test=tf.data.Dataset.from_tensor_slices((Xids_test,Xmask_test))

def map_func(input_ids,mask):
    return {'input_ids':input_ids,'attention_mask':mask}

dataset_test=dataset_test.map(map_func)
# batching it to or the predictions will be multiplied by the shape
dataset_test=dataset_test.batch(64).prefetch(1000)

In [ ]:
def emotion_model():
  input_ids=tf.keras.layers.Input(shape=(max_length,),name='input_ids',dtype='int32')
  input_mask=tf.keras.layers.Input(shape=(max_length,),name='attention_mask',dtype='int32')

  embedding=bert(input_ids,attention_mask=input_mask)[0]
  x=tf.keras.layers.GlobalMaxPool1D()(embedding)
  x=tf.keras.layers.BatchNormalization()(x)
  x=tf.keras.layers.Dense(256,activation='relu')(x)
  x=tf.keras.layers.Dropout(0.2)(x)
  output=tf.keras.layers.Dense(5,activation='softmax')(x)

  model=tf.keras.Model(inputs=[input_ids,input_mask],outputs=output)

  model.layers[2].trainable=False

  model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer='adam',metrics=[tf.keras.metrics.AUC()])

  return model

## Define train and test

In [ ]:
model = emotion_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109927680   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 100,                                         

## Define model and saving path

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(train,
          validation_data=val,
          epochs=8)

In [ ]:
y_pred=model.predict(dataset_test)
y_pred_new = np.argmax(y_pred,axis=1)
y_true = df_test['label_id'].values

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_true, y_pred_new))

In [ ]:
def create_dataset(df):
  Xids_test=np.zeros((df.shape[0],max_length))
  Xmask_test=np.zeros((df.shape[0],max_length))

  for i,sequence in enumerate(df['content']):
    tokens=tokenizer.encode_plus(sequence,max_length=max_length,padding='max_length',add_special_tokens=True,
                           truncation=True,return_token_type_ids=False,return_attention_mask=True,
                           return_tensors='tf')

    Xids_test[i,:] = tokens['input_ids']
    Xmask_test[i,:] = tokens['attention_mask']

  dataset_test=tf.data.Dataset.from_tensor_slices((Xids_test,Xmask_test))

  def map_func(input_ids,mask):
      return {'input_ids':input_ids,'attention_mask':mask}

  dataset_test=dataset_test.map(map_func)
  dataset_test=dataset_test.batch(64).prefetch(1000)

  return dataset_test

df_tails = df_test[df_test.source == "Tails"]
y_true = df_tails['label_id'].values
df_tails = create_dataset(df_tails)

y_pred=model.predict(df_tails)
y_pred_new = np.argmax(y_pred,axis=1)

print(metrics.classification_report(y_true, y_pred_new))